In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Dokumentacja: 

https://numpy.org/doc/2.2/user/basics.indexing.html

#Liczby losowe typu int:

https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html

#Suma liczb:

https://numpy.org/doc/stable/reference/generated/numpy.sum.html

# Zadanie 1. Generowanie Sygnału Sinusoidalnego:
Zadanie: Napisz funkcję do generowania przebiegu sygnału harmonicznego składającego się z wielu sinusów i cosinusów o różnych częstotliwościach i amplitudach.  

In [ ]:
def generate_sinusoidal_signal(t, frequencies, amplitudes, phase_shifts=None):
    """
    Generate a signal composed of multiple sinusoids.
    
    Parameters:
    t : numpy array
        Time array
    frequencies : list or numpy array
        List of frequencies for each component (in Hz)
    amplitudes : list or numpy array
        List of amplitudes for each component
    phase_shifts : list or numpy array, optional
        List of phase shifts for each component (in radians)
        
    Returns:
    signal : numpy array
        Combined sinusoidal signal
    """
    if phase_shifts is None:
        phase_shifts = np.zeros_like(frequencies)
    
    # Initialize signal with zeros
    signal = np.zeros_like(t)
    
    # Add each sinusoidal component
    for f, a, p in zip(frequencies, amplitudes, phase_shifts):
        signal += a * np.sin(2 * np.pi * f * t + p)
    
    return signal

# Parametry sygnału
sampling_rate = 1000  # częstotliwość próbkowania [Hz]
T = 1.0  # czas trwania sygnału [s]

# Generowanie osi czasu
t = np.linspace(0, T, int(T*sampling_rate), endpoint=False)

# Przykładowe użycie
frequencies = [5, 10, 20]  # różne częstotliwości [Hz]
amplitudes = [1.0, 0.5, 0.2]  # różne amplitudy
signal = generate_sinusoidal_signal(t, frequencies, amplitudes)

# Wykres sygnału
plt.figure(figsize=(10, 6))
plt.plot(t, signal)
plt.title('Złożony sygnał sinusoidalny')
plt.xlabel('Czas (s)')
plt.ylabel('Amplituda')
plt.grid(True)
plt.show()

# Zadanie 2: Pierwsza analiza w domenie częstotiwości:

Zaimplementuj funkcję w Pythonie, która wykonuje analizę FFT na podanym sygnale. Następnie wyniki funkcji FFT wyświetl na wykresie. Dodaj podpisy osi x,y itp. 

Zbadaj jak parametry sinusa, typu sampling, długość sygnału, i amplituda poszczególnych sygnałałów wpływa na spektrum FFT. 

In [ ]:
def perform_fft(signal, sampling_rate):
    """
    Perform Fast Fourier Transform on a signal.
    
    Parameters:
    signal : numpy array
        Input signal
    sampling_rate : float
        Sampling frequency in Hz
        
    Returns:
    frequencies : numpy array
        Frequency array
    fft_values : numpy array
        FFT magnitude spectrum
    """
    N = len(signal)
    
    # Perform FFT
    fft_result = np.fft.fft(signal)
    
    # Calculate magnitude spectrum (normalize by dividing by N)
    fft_magnitude = np.abs(fft_result) / N
    
    # For real signal, the spectrum is symmetric - take only the first half
    fft_magnitude = fft_magnitude[:N//2]
    
    # Double all values except DC component (0 Hz) to account for energy in negative frequencies
    fft_magnitude[1:] *= 2
    
    # Calculate frequency array
    frequencies = np.fft.fftfreq(N, 1/sampling_rate)[:N//2]
    
    return frequencies, fft_magnitude

# Używając sygnału z Zadania 1
freq, fft = perform_fft(signal, sampling_rate)

# Wykres widma częstotliwościowego
plt.figure(figsize=(10, 6))
plt.plot(freq, fft)
plt.title('Widmo częstotliwościowe')
plt.xlabel('Częstotliwość (Hz)')
plt.ylabel('Amplituda')
plt.grid(True)
plt.xlim(0, max(frequencies) * 1.2)  # Limit osi x dla lepszej widoczności
plt.show()

# Zadanie 3: Pierwsza analiza w domenie częstotiwości:
Zapoznaj się z dokumentacją funkcji sinc:

https://numpy.org/doc/stable/reference/generated/numpy.sinc.html    
    
Wygeneruj sygnał sin(x)/x - czyli sinc w dziedzinie czasu. Dobierz czas funkcji tak aby zawierał również czas "ujemny" - aby wygeneorwać symetryczną funkcję. 

Następnie przeprowadź analizę sygnału w dziedzinie częstotliwości. Co zauważyłeś?


In [ ]:
# Generowanie funkcji sinc
duration = 10  # sekundy
sampling_rate = 1000  # Hz
t = np.linspace(-duration/2, duration/2, duration * sampling_rate)

# Generowanie sygnału sinc
sinc_signal = np.sinc(2 * t)  # numpy.sinc(x) jest definiowana jako sin(πx)/(πx)

# Wykres sygnału sinc
plt.figure(figsize=(10, 6))
plt.plot(t, sinc_signal)
plt.title('Funkcja Sinc')
plt.xlabel('Czas (s)')
plt.ylabel('Amplituda')
plt.grid(True)
plt.show()

# Wykonanie FFT na sygnale sinc
freq_sinc, fft_sinc = perform_fft(sinc_signal, sampling_rate)

# Wykres widma częstotliwościowego
plt.figure(figsize=(10, 6))
plt.plot(freq_sinc, fft_sinc)
plt.title('Widmo częstotliwościowe funkcji Sinc')
plt.xlabel('Częstotliwość (Hz)')
plt.ylabel('Amplituda')
plt.grid(True)
plt.xlim(0, 5)  # Ograniczenie osi x dla lepszej widoczności
plt.show()

# Zadanie 4: Analiza okien FFT.

Wygeneruj funkcję sinc i następnie zastosuj okno hanninga. Wykonaj analizę FFT dla obu przypadków i porównaj wyniki. Co zauważyłeś? 

In [ ]:
# Generowanie sygnału sinc (z Zadania 3)
duration = 10  # sekundy
sampling_rate = 1000  # Hz
t = np.linspace(-duration/2, duration/2, duration * sampling_rate)
sinc_signal = np.sinc(2 * t)

# Zastosowanie okna Hanninga
hanning_window = np.hanning(len(sinc_signal))
sinc_windowed = sinc_signal * hanning_window

# Wykres obu sygnałów
plt.figure(figsize=(10, 6))
plt.plot(t, sinc_signal, label='Oryginalny Sinc')
plt.plot(t, sinc_windowed, label='Sinc z oknem Hanninga')
plt.title('Funkcja Sinc z i bez okna Hanninga')
plt.xlabel('Czas (s)')
plt.ylabel('Amplituda')
plt.legend()
plt.grid(True)
plt.show()

# Wykonanie FFT dla obu sygnałów
freq_sinc, fft_sinc = perform_fft(sinc_signal, sampling_rate)
freq_windowed, fft_windowed = perform_fft(sinc_windowed, sampling_rate)

# Wykres obu widm częstotliwościowych
plt.figure(figsize=(10, 6))
plt.plot(freq_sinc, fft_sinc, label='FFT oryginalnego Sinc')
plt.plot(freq_windowed, fft_windowed, label='FFT Sinc z oknem Hanninga')
plt.title('Porównanie widm częstotliwościowych')
plt.xlabel('Częstotliwość (Hz)')
plt.ylabel('Amplituda')
plt.legend()
plt.grid(True)
plt.xlim(0, 5)  # Ograniczenie osi x dla lepszej widoczności
plt.show()

# Zadanie 5: Analiza spektralna obrazów z użyciem FFT.

https://scikit-image.org/docs/stable/auto_examples/

https://scikit-image.org/docs/stable/auto_examples/data/plot_3d.html#sphx-glr-auto-examples-data-plot-3d-py

Zapoznaj się z dokumentacją scikit image. Wykonaj analizę spektralną (2D FFT), poszczególnych rysunków. Zastosuj filtry zgodnie ze wskazówkami prowadzącego. 


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from skimage import data

#Dostępne zdjęcia

# 'hubble_deep_field',
# 'immunohistochemistry',
# 'lily',
# 'microaneurysms',
# 'moon',
# 'retina',
# 'shepp_logan_phantom',
# 'skin',
# 'cell',
# 'human_mitosis',

from skimage import data
from skimage.color import rgb2gray
from scipy import ndimage

# Wczytanie obrazu i konwersja do skali szarości, jeśli potrzebne
image = data.hubble_deep_field()
if len(image.shape) > 2:
    image_gray = rgb2gray(image)
else:
    image_gray = image

# Wyświetlenie oryginalnego obrazu
plt.figure(figsize=(16, 12))
plt.subplot(231)
plt.imshow(image)
plt.title('Oryginalny obraz')
plt.axis('off')

# Obliczenie 2D FFT
fft_image = np.fft.fft2(image_gray)
fft_shift = np.fft.fftshift(fft_image)  # Przesunięcie zerowej częstotliwości do środka
magnitude_spectrum = np.log(np.abs(fft_shift) + 1)  # Skala logarytmiczna dla lepszej wizualizacji

# Wyświetlenie FFT
plt.subplot(232)
plt.imshow(magnitude_spectrum, cmap='viridis')
plt.title('Widmo magnitudy 2D FFT')
plt.axis('off')

# Zastosowanie filtra dolnoprzepustowego (zatrzymanie tylko centralnych częstotliwości)
rows, cols = image_gray.shape
crow, ccol = rows//2, cols//2
mask_lowpass = np.zeros((rows, cols), dtype=bool)
r = 50  # promień filtra
mask_lowpass[crow-r:crow+r, ccol-r:ccol+r] = True
fft_filtered_lowpass = fft_shift.copy()
fft_filtered_lowpass[~mask_lowpass] = 0

# Odwrotna FFT do rekonstrukcji obrazu
ifft_filtered_lowpass = np.fft.ifftshift(fft_filtered_lowpass)
img_lowpass = np.fft.ifft2(ifft_filtered_lowpass).real

plt.subplot(233)
plt.imshow(img_lowpass, cmap='gray')
plt.title('Obraz po filtrze dolnoprzepustowym')
plt.axis('off')

# Zastosowanie filtra górnoprzepustowego (usunięcie centralnych częstotliwości)
mask_highpass = ~mask_lowpass
fft_filtered_highpass = fft_shift.copy()
fft_filtered_highpass[~mask_highpass] = 0

# Odwrotna FFT
ifft_filtered_highpass = np.fft.ifftshift(fft_filtered_highpass)
img_highpass = np.fft.ifft2(ifft_filtered_highpass).real

plt.subplot(234)
plt.imshow(img_highpass, cmap='gray')
plt.title('Obraz po filtrze górnoprzepustowym')
plt.axis('off')

# Zastosowanie filtra pasmowo-przepustowego
mask_bandpass = np.zeros((rows, cols), dtype=bool)
r_outer = 80
r_inner = 20
y, x = np.ogrid[:rows, :cols]
mask_area = ((y - crow)**2 + (x - ccol)**2 >= r_inner**2) & ((y - crow)**2 + (x - ccol)**2 <= r_outer**2)
mask_bandpass[mask_area] = True

fft_filtered_bandpass = fft_shift.copy()
fft_filtered_bandpass[~mask_bandpass] = 0

# Wizualizacja maski filtra pasmowo-przepustowego
plt.subplot(235)
plt.imshow(mask_bandpass, cmap='gray')
plt.title('Maska filtra pasmowo-przepustowego')
plt.axis('off')

# Odwrotna FFT
ifft_filtered_bandpass = np.fft.ifftshift(fft_filtered_bandpass)
img_bandpass = np.fft.ifft2(ifft_filtered_bandpass).real

plt.subplot(236)
plt.imshow(img_bandpass, cmap='gray')
plt.title('Obraz po filtrze pasmowo-przepustowym')
plt.axis('off')

plt.tight_layout()
plt.show()